# Creating the 3D model

###### Importing the necessary packages

In [ ]:
from bifacial_radiance import *
import os
from pathlib import Path
import numpy as np
import pandas as pd
from os import listdir

###### Creating a folder and folderstructure (RadianceObj)

In [ ]:
# Defining the path to where the folder is found and the wanted foldername
testfolder = r'C:\bifacial_radiance\...\foldername' 

print ("Your simulation will be stored in %s" % testfolder)

if not os.path.exists(testfolder):
    os.makedirs(testfolder)

# Creating the folderstructure
demo = RadianceObj('PilotprosjektKfb',str(testfolder))

###### Creating the sky

The parameters start_time and end_time are updated for every simulation as the timestamps between these two are the once the irradiance is simulated for.

In [ ]:
start_time = '2023_02_21_2200'   # Startdate on the format YYYY_MM_DD_HHmm
end_time = '2023_02_23_0300'     # Enddate on the format YYYY_MM_DD_HHmm

In [ ]:
# Reading the weather file 
tmy = r'path...\filename.csv'
metdata = demo.readWeatherFile(tmy, starttime = start_time, endtime = end_time, coerce_year = 2023) 

# Setting the ground object by using albedo from the weather file
demo.setGround()

In [ ]:
# Setting the geometry of the tracking system
trackerdict = demo.set1axis(metdata = metdata,       # The weather file 
                            azimuth = 86,            # The azimuth of the test site towards east
                            cumulativesky = False,   # Not using cumulative sky, but gendaylit1axis()
                            fixed_tilt_angle = 90,   # The panels are tilted 90 degrees at all time, fixed tilt
                            axis_azimuth = None)

# Creating the sky files, one for each timestamp
trackerdict = demo.gendaylit1axis()

For the case of cumulative sky, the parameter cumulativesky in demo.set1axis() was set to TRUE, and the command "trackerdict = demo.gendaylit1axis()" was replaced with the command "trackerdict = demo.genCumSky1axis()".

###### Defining the solar panels

In [ ]:
module_type = 'test-module' 

numcellsx = 14        # Number of solar cells in x-direction of one panel
numcellsy = 1         # Number of solar cells in y-direction of one panel
xcell = 0.083         # Width of one solar cell
ycell = 0.165         # Height of one solar cell
xcellgap = 0.002      # The distance between two cells in x-direction
ycellgap = 0          # The distance between two cells in y-direction

xgap = 0.266          # Distance between the two solar panels
ygap = 0              # Distance between two solar cells in y-direction
zgap = 0.005          
numpanels = 1         # Number of solar panels made
glass = True          # Glass coverage around the solar cells in one panel

# Defining the solar cells that consitute one solar panel
cellLevelModuleParams = {'numcellsx': numcellsx, 
                         'numcellsy':numcellsy, 
                         'xcell': xcell, 
                         'ycell': ycell, 
                         'xcellgap': xcellgap, 
                         'ycellgap': ycellgap}

# Defining the solar panel
mymodule = demo.makeModule(name = module_type, 
                           x = 1, 
                           y = 1, 
                           xgap = xgap, 
                           ygap = ygap, 
                           zgap = zgap, 
                           numpanels = numpanels, 
                           glass = glass, 
                           z = 0.0002)

# Adding the solar cells to the solar panel
mymodule.addCellModule(**cellLevelModuleParams)

# Defining origo
originx = 0 
originy = 0 

tilt = 90              # The tilt of the solar panels
pitch = 0.39           # Distance between the rows of panels
hub_height = 0.128     # The distance from the ground to the center of the solar panels
azimuth = 86           # The azimuth of the solar panels towards east
nMods = 4              # Number of solar panels in one row
nRows = 9              # Number of rows with solar panels

# Defining the scene with solar panels
sceneDict = {'tilt':tilt, 
             'pitch':pitch, 
             'hub_height':hub_height, 
             'azimuth':azimuth, 
             'nMods':nMods, 
             'nRows':nRows, 
             'originx':originx, 
             'originy':originy} 

# Creating the scene with solar panels
scene = demo.makeScene(module_type, sceneDict)  

###### Adding glass to the front and the rear side of the solar panels that covers more than only the solar cells

In [ ]:
# Starting positions of the objects in x- and y-direction
x_row = [1.56, 1.17, 0.78, 0.39, 0, -0.39, -0.78, -1.17, -1.56] 
y_row = [-2.04, -0.58, 0.88, 2.35] 
x_row2 = [1.556, 1.166, 0.776, 0.386, -0.004, -0.394, -0.784, -1.174, -1.564] 

# Iterating throug all the indexes (timestamps) in trackerdict to make the object 
for index in trackerdict.keys():
    
    for i, x_val in enumerate(x_row): 
        for j, y_val in enumerate(y_row): 
            
            # Giving the object a unique name
            name = 'GlassF{}{}'.format(i, j)
            
            # Defining the the object with material, size [m] and starting position
            text = '! genbox stock_glass Glass{}{} 0.004 1.35 0.206 | xform -t {} {} 0.025'.format(i, j, x_val, y_val)    

            # Creating the object
            customObj = demo.makeCustomObject(name,text) 
            
            # Appending the object to the scene and rotating it 4 degrees from 90 to 86 degrees
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles,        
                               customObject = customObj, text = "!xform -rz 4") 

    for i, x_val in enumerate(x_row2):
        for j, y_val in enumerate(y_row):
            name = 'GlassF{}{}'.format(i, j)
            text = '! genbox stock_glass Glass{}{} 0.004 1.35 0.206 | xform -t {} {} 0.025'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

###### Adding rails to the panels

In [ ]:
x_row = [-1.8]
y_ende = [-2.07, 3.7]
y_midt = [-0.66, 0.8, 2.26]

for index in trackerdict.keys():
    
    for i, x_val in enumerate(x_row):
        for j, y_val in enumerate(y_ende):
            name = 'Skinneende{}{}'.format(i, j)
            text = '! genbox Metal_Grey Skinneende{}{} 3.608 0.03 0.02 | xform -t {} {} 0.0'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

    for i, x_val in enumerate(x_row):
        for j, y_val in enumerate(y_midt):
            name = 'Skinnemidt{}{}'.format(i, j)
            text = '! genbox Metal_Grey Skinnemidt{}{} 3.608 0.066 0.02 | xform -t {} {} 0.0'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

###### Adding a box to the right side of the glass (from the east side)

In [ ]:
x_row = [1.56, 1.17, 0.78, 0.39, 0, -0.39, -0.78, -1.17, -1.56] 
y_row = [-0.81, 0.65, 2.11, 3.58] 

for index in trackerdict.keys():

    for i, x_val in enumerate(x_row):

        for j, y_val in enumerate(y_row):
            name = 'BoxF{}{}'.format(i, j)
            text = '! genbox black Box{}{} 0.025 0.064 0.064 | xform -t {} {} 0.128'.format(i, j, x_val, y_val) 

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

###### Adding L-shaped metal between the rails and panels

In [ ]:
i = 0.03 

x_row = [1.54, 1.15, 0.76, 0.37, -0.02, -0.41, -0.80, -1.19, -1.58] 
y_rowv = [-2.04-i, -0.58-i, 0.88-i, 2.35-i] 
y_rowh = [-0.75+i, 0.71+i, 2.17+i, 3.64+i]
y_row = [-2.04, -0.69, -0.58, 0.77, 0.88, 2.23, 2.35, 3.7]

for index in trackerdict.keys():

    for i, x_val in enumerate(x_row):

        for j, y_val in enumerate(y_rowv):
            name = 'L_shapeB{}{}'.format(i, j)
            text = '! genbox black L_shapeB{}{} 0.039 0.07 0.003 | xform -t {} {} 0.02'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

        for j, y_val in enumerate(y_rowh):
            name = 'L_shapeBV{}{}'.format(i, j)
            text = '! genbox black L_shapeBV{}{} 0.039 0.07 0.003 | xform -t {} {} 0.02'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

        for j, y_val in enumerate(y_row):
            name = 'L_shapeT{}{}'.format(i, j)
            text = '! genbox black L_shapeT{}{} 0.039 0.003 0.08 | xform -t {} {} 0.02'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

###### Adding reference cells to the PV system

In [ ]:
x_row19 = [1.55, -1.57]
x_row5 = [0]
y_row19 = [2.25]
y_row5 = [0.78]

for index in trackerdict.keys():
    
    for i, x_val in enumerate(x_row19):
        for j, y_val in enumerate(y_row19):
            name = 'Referansecelle{}{}'.format(i, j)
            text = '! genbox black Referansecelle{}{} 0.03 0.079 0.119 | xform -t {} {} 0.112'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

    for i, x_val in enumerate(x_row5):
        for j, y_val in enumerate(y_row5):
            name = 'ReferansecelleM{}{}'.format(i, j)
            text = '! genbox black ReferansecelleM{}{} 0.03 0.079 0.119 | xform -t {} {} 0.112'.format(i, j, x_val, y_val)

            customObj = demo.makeCustomObject(name,text)
            demo.appendtoScene(radfile = trackerdict[index]['scene'].radfiles, 
                               customObject = customObj, text = "!xform -rz 4")

###### Making an octfile - combining the ground, the sky and the scene objects

In [ ]:
octfile = demo.makeOct1axis(trackerdict)

# Simulating the irradiance on the three reference cell positions

In [ ]:
analysis = AnalysisObj(octfile, demo.basename) 

###### Simulating irradiance on reference cell 1

In [ ]:
# Looping through the keys of trackerdict
for timess in trackerdict.keys():    
    
    # Doing a front- and backscan 
    frontscan, backscan = analysis.moduleAnalysis(trackerdict[timess]['scene']) 
    
    frontscan1 = frontscan
    backscan1 = backscan
    
    frontscan1['xstart'] = 1.46     # Position of sensor in x-direction on the east side of reference cell 1
    frontscan1['ystart'] = 2.39     # Position of sensor in y-direction on the east side of reference cell 1
    frontscan1['zstart'] = 0.18     # Position of sensor in z-direction on the east side of reference cell 1
    
    frontscan1['Nx'] = 1            # Number of sensors in x-direction
    frontscan1['Ny'] = 1            # Number of sensors in y-direction
    frontscan1['Nz'] = 1            # Number of sensors in z-direction
    
    backscan1['xstart'] = 1.34      # Position of sensor in x-direction on the west side of reference cell 1
    backscan1['ystart'] = 2.38      # Position of sensor in y-direction on the west side of reference cell 1
    backscan1['zstart'] = 0.18      # Position of sensor in z-direction on the west side of reference cell 1
    
    backscan1['Nx'] = 1
    backscan1['Ny'] = 1
    backscan1['Nz'] = 1
    
    # Simulating the irradiance for the given sensor positions and saving the results 
    results = analysis.analysis(trackerdict[timess]['octfile'], demo.basename + '_referansecelle1_'+str(timess), 
                                frontscan1, backscan1)

In [ ]:
dates1 = []      # Creating an empty list for the dates
values1 = []     # Creating an empty list for the irradiance on the east side (front) of the reference cell
values11 = []    # Creating an empty list for the irradiance on the west side (back) of the reference cell

for filename in listdir(testfolder + r'\results'):  # Looping thorough the files in testfolder
    if '_referansecelle1_' in filename:             # If '_referansecelle1_' is in the filename the code below is runned
        print(filename) 
            
        year = filename[37:41]     # Selecting the letterpositions for the year
        day = filename[45:47]      # Selecting the letterpositions for the day
        month = filename[42:44]    # Selecting the letterpositions for the month
        hour = filename[48:50]     # Selecting the letterpositions for the hour

        dates1.append((pd.to_datetime('{}-{}-{} {}:00:00'.format(year, month, day, hour))))  # Adding the dates to dates1
        temp1_df = pd.read_csv(testfolder+'\\results\\'+filename)                            # Reading the irradiance results
        values1.append(temp1_df['Wm2Front'].values[0])                                       # Adding the irradiance to values1
        values11.append(temp1_df['Wm2Back'].values[0])                                       # Adding the irradiance to values11

# Crating a dataframe with the dates as index and columns including the irradiance on the east side as front and irradiance
# on the west side as back and then forcing the frequency to be an hour so timestamps without light are also included, but
# filled with NaN values
data1_df_temp = pd.DataFrame(index = dates1, data = {'front': values1, 'back': values11}) 
data1_df = data1_df_temp.asfreq('60T')

###### Simulating irradiance on reference cell 2

In [ ]:
for timess in trackerdict.keys():
    
    frontscan, backscan = analysis.moduleAnalysis(trackerdict[timess]['scene'])
    
    frontscan2 = frontscan
    backscan2 = backscan
    
    frontscan2['xstart'] = -0.01
    frontscan2['ystart'] = 0.82
    frontscan2['zstart'] = 0.18
    
    frontscan2['Nx'] = 1
    frontscan2['Ny'] = 1
    frontscan2['Nz'] = 1
    
    backscan2['xstart'] = -0.12
    backscan2['ystart' ] = 0.79
    backscan2['zstart'] = 0.18
    
    backscan2['Nx'] = 1
    backscan2['Ny'] = 1
    backscan2['Nz'] = 1
    
    results = analysis.analysis(trackerdict[timess]['octfile'], demo.basename + '_referansecelle2_'+str(timess), 
                                frontscan2, backscan2)

In [ ]:
dates2 = []
values2 = []
values22 = []

for filename in listdir(testfolder + r'\results'):
    if '_referansecelle2_' in filename: 
        print(filename)
        
        year = filename[37:41]
        day = filename[45:47]
        month = filename[42:44]
        hour = filename[48:50]   
            
        dates2.append((pd.to_datetime('{}-{}-{} {}:00:00'.format(year, month, day, hour))))
        temp2_df = pd.read_csv(testfolder+'\\results\\'+filename)
        values2.append(temp2_df['Wm2Front'].values[0])
        values22.append(temp2_df['Wm2Back'].values[0])
        
data2_df_temp = pd.DataFrame(index = dates2, data = {'front': values2, 'back': values22})
data2_df = data2_df_temp.asfreq('60T')

###### Simulating irradiance on reference cell 3

In [ ]:
for timess in trackerdict.keys():
    
    frontscan, backscan = analysis.moduleAnalysis(trackerdict[timess]['scene'])

    frontscan3 = frontscan
    backscan3 = backscan
    
    frontscan3['xstart'] = -1.67
    frontscan3['ystart'] = 2.17
    frontscan3['zstart'] = 0.18
    
    frontscan3['Nx'] = 1
    frontscan3['Ny'] = 1
    frontscan3['Nz'] = 1
    
    backscan3['xstart'] = -1.77
    backscan3['ystart' ] = 2.17
    backscan3['zstart'] = 0.18
    
    backscan3['Nx'] = 1
    backscan3['Ny'] = 1
    backscan3['Nz'] = 1
    
    results = analysis.analysis(trackerdict[timess]['octfile'], demo.basename + '_referansecelle3_'+str(timess), 
                                frontscan3, backscan3)

In [ ]:
dates3 = []
values3 = []
values33 = []

for filename in listdir(testfolder + r'\results'):
    if '_referansecelle3_' in filename:
        print(filename)
        
        year = filename[37:41]
        day = filename[45:47]
        month = filename[42:44]
        hour = filename[48:50]  
           
        dates3.append((pd.to_datetime('{}-{}-{} {}:00:00'.format(year, month, day, hour))))
        temp3_df = pd.read_csv(testfolder+'\\results\\'+filename)
        values3.append(temp3_df['Wm2Front'].values[0])
        values33.append(temp3_df['Wm2Back'].values[0])
    
data3_df_temp = pd.DataFrame(index = dates3, data = {'front': values3, 'back':values33})
data3_df = data3_df_temp.asfreq('60T')

###### Saving the irradiance results in wanted folder with a filename

In [ ]:
data1_df.to_csv(r'path...\filename.csv')
data2_df.to_csv(r'path...\filename.csv')
data3_df.to_csv(r'path...\filename.csv')

The steps where data1_df, data2_df, and data3_df were skipped when simulating cumulative sky, instead the results in each for-loop when doing front- and backscan was saved because cumulative sky gives one irradiance value on the east and west side of the reference cell for the given time period and therefore, do not require a frequency to force NaN values when there is no light.